In [15]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display

from atj_trading.backtester import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [6]:
symbol = 'EURUSD'
start_dt = datetime(2023, 1, 1)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)

sma_period = 20
ohlc['sma_20'] = ohlc['open'].rolling(sma_period).mean()

display(ohlc)

fig = create_price_fig(ohlc, indicators=['sma_20'])
display(fig)

,time,open,high,low,close,sma_20
0,2023-01-02,1.06950,1.07058,1.06494,1.06577,NaN
1,2023-01-03,1.06577,1.06831,1.05196,1.05488,NaN
2,2023-01-04,1.05483,1.06352,1.05405,1.06038,NaN
3,2023-01-05,1.06038,1.06316,1.05151,1.05217,NaN
4,2023-01-06,1.05217,1.06479,1.04820,1.06454,NaN
...,...,...,...,...,...,...
413,2024-08-05,1.09217,1.10091,1.08927,1.09507,1.085751
414,2024-08-06,1.09467,1.09631,1.09038,1.09310,1.086372
415,2024-08-07,1.09228,1.09367,1.09055,1.09223,1.086924
416,2024-08-08,1.09180,1.09452,1.08816,1.09182,1.087379


In [8]:
def get_signal(x):
    if x['open'] > x['sma_20']:
        return 'buy'
    elif x['open'] < x['sma_20']:
        return 'sell'
    
ohlc['signal'] = ohlc.apply(get_signal, axis=1)
ohlc

,time,open,high,low,close,sma_20,signal
0,2023-01-02,1.06950,1.07058,1.06494,1.06577,NaN,None
1,2023-01-03,1.06577,1.06831,1.05196,1.05488,NaN,None
2,2023-01-04,1.05483,1.06352,1.05405,1.06038,NaN,None
3,2023-01-05,1.06038,1.06316,1.05151,1.05217,NaN,None
4,2023-01-06,1.05217,1.06479,1.04820,1.06454,NaN,None
...,...,...,...,...,...,...,...
413,2024-08-05,1.09217,1.10091,1.08927,1.09507,1.085751,buy
414,2024-08-06,1.09467,1.09631,1.09038,1.09310,1.086372,buy
415,2024-08-07,1.09228,1.09367,1.09055,1.09223,1.086924,buy
416,2024-08-08,1.09180,1.09452,1.08816,1.09182,1.087379,buy


In [10]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 100000 # 1 lots
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        orders.open_trade(symbol, volume, 'sell')
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data['signal'] == 'sell':
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data['signal'] == 'buy':
            orders.close_trade(trade)

In [12]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,profit,commission,profit_net,profit_cumulative
0,closed,EURUSD,buy,100000,2023-01-27 00:00:00,1.08912,2023-02-06 00:00:00,1.07816,0,0,-1096.0,-7.0,-1103.0,-1103.0
1,closed,EURUSD,sell,100000,2023-02-07 00:00:00,1.07251,2023-03-07 00:00:00,1.06827,0,0,424.0,-7.0,417.0,-686.0
2,closed,EURUSD,sell,100000,2023-03-08 00:00:00,1.05489,2023-03-13 00:00:00,1.0681,0,0,-1321.0,-7.0,-1328.0,-2014.0
3,closed,EURUSD,buy,100000,2023-03-14 00:00:00,1.07311,2023-03-16 00:00:00,1.05753,0,0,-1558.0,-7.0,-1565.0,-3579.0
4,closed,EURUSD,sell,100000,2023-03-17 00:00:00,1.06129,2023-03-20 00:00:00,1.0683,0,0,-701.0,-7.0,-708.0,-4287.0
5,closed,EURUSD,buy,100000,2023-03-21 00:00:00,1.07236,2023-05-10 00:00:00,1.09615,0,0,2379.0,-7.0,2372.0,-1915.0
6,closed,EURUSD,sell,100000,2023-05-11 00:00:00,1.09824,2023-06-09 00:00:00,1.07829,0,0,1995.0,-7.0,1988.0,73.0
7,closed,EURUSD,sell,100000,2023-06-12 00:00:00,1.07437,2023-06-13 00:00:00,1.07577,0,0,-140.0,-7.0,-147.0,-74.0
8,closed,EURUSD,buy,100000,2023-06-14 00:00:00,1.07927,2023-07-06 00:00:00,1.08531,0,0,604.0,-7.0,597.0,523.0
9,closed,EURUSD,buy,100000,2023-07-07 00:00:00,1.08911,2023-07-28 00:00:00,1.09762,0,0,851.0,-7.0,844.0,1367.0


In [13]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [14]:
backtest_fig = bt.visualize_backtest(ohlc, indicators=['sma_20'])
backtest_fig

In [16]:
evaluate_backtest(bt.trades)

biggest_profit 2449.0
biggest_loss -1558.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,profit,commission,profit_net,profit_cumulative
1,closed,EURUSD,sell,100000,2023-02-07,1.07251,2023-03-07,1.06827,0,0,424.0,-7.0,417.0,-686.0
5,closed,EURUSD,buy,100000,2023-03-21,1.07236,2023-05-10,1.09615,0,0,2379.0,-7.0,2372.0,-1915.0
6,closed,EURUSD,sell,100000,2023-05-11,1.09824,2023-06-09,1.07829,0,0,1995.0,-7.0,1988.0,73.0
8,closed,EURUSD,buy,100000,2023-06-14,1.07927,2023-07-06,1.08531,0,0,604.0,-7.0,597.0,523.0
9,closed,EURUSD,buy,100000,2023-07-07,1.08911,2023-07-28,1.09762,0,0,851.0,-7.0,844.0,1367.0
10,closed,EURUSD,sell,100000,2023-07-31,1.10194,2023-08-31,1.09228,0,0,966.0,-7.0,959.0,2326.0
11,closed,EURUSD,sell,100000,2023-09-01,1.08434,2023-10-11,1.05985,0,0,2449.0,-7.0,2442.0,4768.0
16,closed,EURUSD,buy,100000,2023-10-30,1.05643,2023-11-02,1.0565,0,0,7.0,-7.0,0.0,2440.0
17,closed,EURUSD,buy,100000,2023-11-03,1.06185,2023-12-05,1.08341,0,0,2156.0,-7.0,2149.0,4589.0
20,closed,EURUSD,buy,100000,2024-01-11,1.09665,2024-01-12,1.09686,0,0,21.0,-7.0,14.0,3126.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,profit,commission,profit_net,profit_cumulative
0,closed,EURUSD,buy,100000,2023-01-27,1.08912,2023-02-06,1.07816,0,0,-1096.0,-7.0,-1103.0,-1103.0
2,closed,EURUSD,sell,100000,2023-03-08,1.05489,2023-03-13,1.0681,0,0,-1321.0,-7.0,-1328.0,-2014.0
3,closed,EURUSD,buy,100000,2023-03-14,1.07311,2023-03-16,1.05753,0,0,-1558.0,-7.0,-1565.0,-3579.0
4,closed,EURUSD,sell,100000,2023-03-17,1.06129,2023-03-20,1.0683,0,0,-701.0,-7.0,-708.0,-4287.0
7,closed,EURUSD,sell,100000,2023-06-12,1.07437,2023-06-13,1.07577,0,0,-140.0,-7.0,-147.0,-74.0
12,closed,EURUSD,buy,100000,2023-10-12,1.0609,2023-10-13,1.05269,0,0,-821.0,-7.0,-828.0,3940.0
13,closed,EURUSD,sell,100000,2023-10-16,1.05103,2023-10-18,1.05698,0,0,-595.0,-7.0,-602.0,3338.0
14,closed,EURUSD,sell,100000,2023-10-19,1.05318,2023-10-20,1.05791,0,0,-473.0,-7.0,-480.0,2858.0
15,closed,EURUSD,buy,100000,2023-10-23,1.05936,2023-10-27,1.05525,0,0,-411.0,-7.0,-418.0,2440.0
18,closed,EURUSD,sell,100000,2023-12-06,1.07891,2023-12-14,1.08716,0,0,-825.0,-7.0,-832.0,3757.0


avg_win 950.0666666666667
avg_loss -722.4444444444445
count_profit_trades 15
count_loss_trades 18
win_rate 0.83
rrr 1.32


,order_type,profit
0,buy,-195.0
1,sell,1442.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,profit,commission,profit_net,profit_cumulative,dayofweek
0,closed,EURUSD,buy,100000,2023-01-27,1.08912,2023-02-06,1.07816,0,0,-1096.0,-7.0,-1103.0,-1103.0,0
1,closed,EURUSD,sell,100000,2023-02-07,1.07251,2023-03-07,1.06827,0,0,424.0,-7.0,417.0,-686.0,1
2,closed,EURUSD,sell,100000,2023-03-08,1.05489,2023-03-13,1.0681,0,0,-1321.0,-7.0,-1328.0,-2014.0,0
3,closed,EURUSD,buy,100000,2023-03-14,1.07311,2023-03-16,1.05753,0,0,-1558.0,-7.0,-1565.0,-3579.0,3
4,closed,EURUSD,sell,100000,2023-03-17,1.06129,2023-03-20,1.0683,0,0,-701.0,-7.0,-708.0,-4287.0,0
5,closed,EURUSD,buy,100000,2023-03-21,1.07236,2023-05-10,1.09615,0,0,2379.0,-7.0,2372.0,-1915.0,2
6,closed,EURUSD,sell,100000,2023-05-11,1.09824,2023-06-09,1.07829,0,0,1995.0,-7.0,1988.0,73.0,4
7,closed,EURUSD,sell,100000,2023-06-12,1.07437,2023-06-13,1.07577,0,0,-140.0,-7.0,-147.0,-74.0,1
8,closed,EURUSD,buy,100000,2023-06-14,1.07927,2023-07-06,1.08531,0,0,604.0,-7.0,597.0,523.0,3
9,closed,EURUSD,buy,100000,2023-07-07,1.08911,2023-07-28,1.09762,0,0,851.0,-7.0,844.0,1367.0,4


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,profit,commission,profit_net,profit_cumulative,dayofweek,current_max,drawdown
0,closed,EURUSD,buy,100000,2023-01-27,1.08912,2023-02-06,1.07816,0,0,-1096.0,-7.0,-1103.0,-1103.0,0,-1103.0,0.0
1,closed,EURUSD,sell,100000,2023-02-07,1.07251,2023-03-07,1.06827,0,0,424.0,-7.0,417.0,-686.0,1,-686.0,0.0
2,closed,EURUSD,sell,100000,2023-03-08,1.05489,2023-03-13,1.0681,0,0,-1321.0,-7.0,-1328.0,-2014.0,0,-686.0,-1328.0
3,closed,EURUSD,buy,100000,2023-03-14,1.07311,2023-03-16,1.05753,0,0,-1558.0,-7.0,-1565.0,-3579.0,3,-686.0,-2893.0
4,closed,EURUSD,sell,100000,2023-03-17,1.06129,2023-03-20,1.0683,0,0,-701.0,-7.0,-708.0,-4287.0,0,-686.0,-3601.0
5,closed,EURUSD,buy,100000,2023-03-21,1.07236,2023-05-10,1.09615,0,0,2379.0,-7.0,2372.0,-1915.0,2,-686.0,-1229.0
6,closed,EURUSD,sell,100000,2023-05-11,1.09824,2023-06-09,1.07829,0,0,1995.0,-7.0,1988.0,73.0,4,73.0,0.0
7,closed,EURUSD,sell,100000,2023-06-12,1.07437,2023-06-13,1.07577,0,0,-140.0,-7.0,-147.0,-74.0,1,73.0,-147.0
8,closed,EURUSD,buy,100000,2023-06-14,1.07927,2023-07-06,1.08531,0,0,604.0,-7.0,597.0,523.0,3,523.0,0.0
9,closed,EURUSD,buy,100000,2023-07-07,1.08911,2023-07-28,1.09762,0,0,851.0,-7.0,844.0,1367.0,4,1367.0,0.0


ValueError: Plotly Express cannot process wide-form data with columns of different type.